### Dataset & DataLoader 살펴보기
- Pytorch에서 배치크기만 데이터를 조절하기 위한 메커니즘
- Dataset : 사용 데이터를 기반으로 사용자 정의 클래스 작성 
- DataLoader : 지정된 Datase에서 지정된 배치사이즈 만큼 피쳐와 타겟을 추출하여 전달

In [3]:
# 모듈 로딩
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

In [4]:
# 데이터 준비
x_data = torch.IntTensor([[10, 20, 30],[20, 30, 40],[30, 40, 50],[40, 50, 60], [50, 60, 70]])
y_data = torch.IntTensor([[20], [30], [40], [50], [60]])

#  행 길이는 꼭 맞춰줘야함!! 

print(f"x_data => {x_data.shape} {x_data.ndim}D")
print(f"y_data => {y_data.shape} {y_data.ndim}D")

x_data => torch.Size([5, 3]) 2D
y_data => torch.Size([5, 1]) 2D


### 2. 데이터셋 생성 

### 2-1 TensorDataset 활용 : Dataset의 sub_class 

In [5]:
# tensorDataset 클래스 로딩
from torch.utils.data import TensorDataset

In [6]:
dataset = TensorDataset(x_data, y_data) #(입력데이터, 출력데이터)
dataset # 데이터를 쌍으로 묶어서 저장.

In [7]:
dataset.tensors

(tensor([[10, 20, 30],
         [20, 30, 40],
         [30, 40, 50],
         [40, 50, 60],
         [50, 60, 70]], dtype=torch.int32),
 tensor([[20],
         [30],
         [40],
         [50],
         [60]], dtype=torch.int32))

In [8]:
## 2-2 사용자 정의 데이터셋 생성

In [9]:
### data 준비
file = '../data/iris.csv'

irisDF = pd.read_csv(file)
irisDF

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [10]:
# 넘파이로도 한번 열어보자
irisNP = np.loadtxt(file, delimiter="," ,skiprows=1, usecols=[0, 1, 2, 3])
irisNP

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [11]:
irisDF.__class__.__name__, irisNP.__class__.__name__

('DataFrame', 'ndarray')

In [12]:
isinstance(irisDF, pd.DataFrame), isinstance(irisNP, np.ndarray), isinstance([1, 2], list)

(True, True, True)

In [15]:
# 사용자 정의 DataSet Class
# Dataset = 부모 
# DLDataset = 자식
# 데이터의 텐서변환

class DLDataset(Dataset):
    # 초기화 함수
    def __init__(self, x_data, y_data):
        super().__init__()
        # x, y 데이터 => ndarray
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data 
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data 
        # 데이터 프레임이라면 values로 arrary 뽑아줘야함 

        # ndarray => tensor 
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.LongTensor(y_data)

    # 데이터셋의 개수 체크 함수
    def __len__(self):
        return self.target.shape[0]
    
    # 특정 인덱스 데이터 + 라벨 반환 callback function 
    def __getitem__(self, index):
        return self.feature[index], self.target[index] # 둘 다 튜플로 반환.

In [16]:
# feature와 label 분리 
featureDF = irisDF[irisDF.columns[:-1]]
targetNP = irisDF[irisDF.columns[-1]]

from sklearn.preprocessing import LabelEncoder # object 타입 타겟 => int 타입 타겟 변환 

print(f"featureDF => {featureDF.shape}, {featureDF.ndim}D")
print(f"targetDF => {targetNP.shape}, {targetNP.ndim}D")

targetNP = LabelEncoder().fit_transform(targetNP).reshape(-1, 1)
print(f"targetDF => {targetNP.shape}, {targetNP.ndim}D")

featureDF => (150, 4), 2D
targetDF => (150,), 1D
targetDF => (150, 1), 2D


In [17]:
my_dataset = DLDataset(featureDF, targetNP) # DF, numpy

In [18]:
my_dataset[0], featureDF.iloc[0], targetNP[0]

((tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0])),
 sepal.length    5.1
 sepal.width     3.5
 petal.length    1.4
 petal.width     0.2
 Name: 0, dtype: float64,
 array([0]))

In [19]:
my_dataset2 = DLDataset(irisNP, targetNP) # numpy, numpy # irisNP => 피처만 갖고 온 것.

In [20]:
## 2-3 학습용, 검증용, 테스트용 Dataset

In [22]:
### => 파이토치
from torch.utils.data import random_split


# 데이터 비율 정하기 
seed = torch.Generator().manual_seed(42)
trainDS, validDS, testDS = random_split(my_dataset2, [0.7, 0.1, 0.2], generator=seed) 
# 개수로 주던가 비율로 주던가

print(f"trainDS => {len(trainDS)}개, validDS => {len(validDS)}개, testDS => {len(testDS)}개")

trainDS => 105개, validDS => 15개, testDS => 30개


In [23]:
print(f"Subset 속성 => {trainDS.indices}\ndataset => {trainDS.dataset}")

Subset 속성 => [42, 95, 30, 64, 52, 35, 130, 40, 82, 17, 108, 94, 68, 97, 117, 127, 41, 44, 57, 140, 149, 32, 23, 102, 16, 113, 71, 18, 67, 66, 0, 25, 101, 112, 91, 3, 59, 116, 86, 84, 106, 142, 43, 39, 26, 98, 93, 20, 87, 19, 120, 114, 7, 63, 76, 89, 36, 45, 37, 56, 58, 122, 51, 145, 24, 21, 105, 62, 15, 11, 48, 133, 88, 50, 6, 134, 111, 8, 49, 75, 69, 124, 4, 147, 80, 100, 99, 141, 47, 107, 13, 109, 129, 28, 38, 53, 121, 5, 55, 31, 73, 74, 54, 29, 12]
dataset => <__main__.DLDataset object at 0x0000021B5582C130>


전체 데이터 셋 => 학습DS, 테스터DS, 검증DF
- 속성 : index가 저장되어 있음 

In [24]:
## DataLoader 생성 : 학습용, 검증용, 테스트용
# drop_last 매개변수 : 배치사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 설정 (default = False) True하면 걍 버리는 거임(valid로 쓸 데이터 없는 경우 버린 걸 사용하는 경우가 있음 )
batch_size = 5
trainDL = DataLoader(trainDS, batch_size=batch_size)
validDL = DataLoader(validDS, batch_size=batch_size)
testDL = DataLoader(testDS, batch_size=batch_size)

len(trainDL), len(validDL), len(testDL) 
# 배치사이즈 만큼 나눠진 수가 나오는 것.
# 즉, 1에포크 당 반복을 몇번하는지 알 수 있는 것 

(21, 3, 6)

In [25]:
# Epoch당 반복 단위
print(f"Batch_size : {batch_size}")
print(f"trainDS => {len(trainDS)}개, validDS => {len(validDS)}개, testDS => {len(testDS)}개")
print(f"trainDL => {len(trainDL)}개, validDL => {len(validDL)}개, testDL => {len(testDL)}개")

Batch_size : 5
trainDS => 105개, validDS => 15개, testDS => 30개
trainDL => 21개, validDL => 3개, testDL => 6개


In [26]:
# DataLoader 생성
for _, (feature, target) in enumerate(trainDL):
    print(f"[{_}] feature {feature.shape}")
    ## 로더에서 가지고 온 데이터 만큼 학습 진행 
    ## 학습 후에는 검증도 할 것! 

[0] feature torch.Size([5, 4])
[1] feature torch.Size([5, 4])
[2] feature torch.Size([5, 4])
[3] feature torch.Size([5, 4])
[4] feature torch.Size([5, 4])
[5] feature torch.Size([5, 4])
[6] feature torch.Size([5, 4])
[7] feature torch.Size([5, 4])
[8] feature torch.Size([5, 4])
[9] feature torch.Size([5, 4])
[10] feature torch.Size([5, 4])
[11] feature torch.Size([5, 4])
[12] feature torch.Size([5, 4])
[13] feature torch.Size([5, 4])
[14] feature torch.Size([5, 4])
[15] feature torch.Size([5, 4])
[16] feature torch.Size([5, 4])
[17] feature torch.Size([5, 4])
[18] feature torch.Size([5, 4])
[19] feature torch.Size([5, 4])
[20] feature torch.Size([5, 4])


##### 4. Model 클래스 정의 : 입출력 feature 수, 층 수, 은닉층의 노드 수 
    * 입력층 : 입력 <= 피쳐 개수, iris 4개
    * 은닉층 : 마음대로 알아서 잘 
    * 출력층 : 출력 <= 분류 = 타겟 클래스 수만큼 / 회귀 = 무조건 1개 

In [27]:
import pandas
import numpy as np

import torch
import torch.nn as nn # linear Regression 기능의 클래스 Linear
import torch.nn.functional as F # 손실함수
import torch.optim as optim # 최적화

In [28]:
class CModel(nn.Module):
    # 구성요소 정의 함수
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.input_layer = nn.Linear(in_dim, 100) # in_dim ==> 컬럼 수.
        self.relu = nn.ReLU() # 활성화 함수 
        self.hidden_layer = nn.Linear(100, 27)
        self.output_layer = nn.Linear(27, out_dim)


In [ ]:
# 5. 학습 준비 : 실행디바이스, 모델, 최적화, 손실함수, 학습 횟수, 학습 함수, 예측 함수

In [29]:
# 실행 디바이스 설정
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" # GPU있으면 쓰고 없으면 cpu로 

# 학습 횟수
EPOCHS = 50

In [30]:
# 모델 인스턴스
IN, OUT = my_dataset2.feature.shape[1], len(my_dataset2.target.unique()) # in은 피처의 컬럼 수/ out은 target의 고유값 수
model = CModel(IN, OUT).to(DEVICE)

In [31]:
my_dataset2.feature.shape[1], len(my_dataset2.target.unique())

(4, 3)

In [32]:
# 손실함수 인스턴스 
LOSS_FN = nn.CrossEntropyLoss().to(DEVICE) # 다중분류니까

# 최적화 인스턴스
OPTIMIZER = optim.Adam(model.parameters())